# Final Report
## 
* Brian Oneill
* Scott Barnett
* Keila Camarillo
* Esayas Asefa

* 11 June 2023

## Goals
* 
*  
* 
* 

## Imports

In [1]:
import warnings
warnings.filterwarnings("ignore")
import wrangle as w

import pandas as pd
import numpy as np

#splits
from sklearn.model_selection import train_test_split

#visualization
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns

#scaling
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.metrics import precision_score, accuracy_score, recall_score, classification_report

#model
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.linear_model import LogisticRegression


In [2]:
df = pd.read_csv('merged_data.csv')

## Acquire

* Data acquired from the Codeup Database and the provided * document 
    *
* It contained ** * rows and * columns**
* The data was aquired on **10 JULY 2023**
* Each row represents a page access of the Codeup lesson server
* Each column represents a feature of the access event


## Prepare



In [3]:
df, model_df = w.prep_df(df)

In [4]:
df.head()

,dob,outcome,sex,name,intake_type,condition,species,breed,outcome_age,rel_month,rel_year,primary_color,is_tabby,mix_color
0,2017-05-02,adoption,neutered male,1,owner surrender,normal,cat,mix,736,may,2019,brown,1,1
1,2017-07-12,adoption,neutered male,1,stray,normal,dog,mix,371,jul,2018,white,0,1
5,2014-03-12,transfer,intact male,1,stray,medical attention,cat,mix,6,mar,2014,orange,1,0
8,2018-03-01,adoption,neutered male,1,owner surrender,normal,dog,two breeds,797,may,2020,white,0,1
9,2022-05-04,adoption,neutered male,1,stray,normal,cat,mix,76,jul,2022,brown,1,0


In [8]:
model_df.head()

,name,outcome_age,is_tabby,mix_color,outcome_other,outcome_transfer,sex_intact male,sex_neutered male,sex_spayed female,intake_type_euthanasia request,...,primary_color_gray,primary_color_lilac point,primary_color_lynx point,primary_color_orange,primary_color_red,primary_color_sable,primary_color_seal point,primary_color_tan,primary_color_tricolor,primary_color_white
0,1,736,1,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,371,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
5,1,6,1,0,0,1,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
8,1,797,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
9,1,76,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


### Data Dictionary

|Feature   |Datatype| Unit       |Description   |
|----------|--------|------------|--------------|
|Animal ID | String |Alphanumeric | Unique number assigned to each animal when their record is created in the database |
|Animal Type| String| Alphanumeric|Animal category: dog, cat, wildlife, other, etc.  |
|Activity Number|Unique number assigned to an activity related to a service request.|
|Activity Sequence|Sequence starts with 1 usually then a follow up sequence is created until activity is completed.|
|Census Tract| | |Government census tract in which the action was located.|
|Council District|City of Dallas Council District in which the action was located.|
|DateTime| | | | |Date and time of action  |
|MonthYear|Month and year of aciton|
|Kennel Number|Location of the animal at the time of the report|
|Kennel Status|Availability of the animal.|
|Intake Type|Type or purpose of intake; used primarily to analyze intake trends.|
|Intak Total|Additional categorization of purpose of intake; used primarily to analyze intake trends.|
|Reason|Reason the animal was surrendered or taken in.|
|Staff Id|Unique ID number assigned to the staff person who entered the record.|
|Intake Date|Date the animal was intaken by DAS.|
|Intake Time|Time the animal was intaken by DAS.|
|Due Out|Date the animal's stray hold expires and animal will be available for non-return to owner outcomes; date DAS has full ownership of the animal based on city ordinance.|
|Intake Condition|Apparent medical condition of the animal when it was taken in by DAS.|
|Hold Request|Routing or pathway identified for the animal at the time of the report. Pathways are used to move animals towards the outcome management recommends at the time based on behavior, medical condition, and history. Pathways are reviewed and updated frequently as an animal's behavior or medical condition changes.|
|Outcome Type|Final outcome of the animal if they are no longer under the care of DAS at the time of the report.|
|Outcome Subtype|Additional details on the outcome of the animal used primiarly for outcome trend analysis.|
|Outcome Date| Date the animal was outcomed by DAS / left DAS' care.|
|Outcome Time|Time the animal was outcomed by DAS / left DAS' care.|
|Receipt number|Unique number assigned to each financial transaction that occurs in Chameleon database.|
|Impound Number|Unique number assigned to each impound performed by DAS staff; each impound can include multiple animals.|
|Service Request Number|Unique number assigned to each impound performed by DAS staff; each impound can include multiple animals.|
|Outcome condition|Apparent medical condition of the animal when it was released from DAS.|
|Chip status|Notates whether staff were successful in scanning animal for a microchip.|
|Animal origin| Notates whether the animal came in through DAS' Pet Support Lobby (Over the Counter) or through Field Services (Field).|
|Additional Information|Additional staff notes.|
|Month|Month the record was created.|
|Year |City of Dallas Fiscal Year the record was created.|
|Date of Birth | Birth date of the animal  |
|Sex upon outcome | Whether the animal was neutered/spayed during outcome  |
|Age upon outcome | Age of animal at time of outcome|
|Breed |Breed of animal|
|Color |The color of the animal|

### Bring in Data for Exploration

In [5]:
# Bringing in data and prparing it for exploration
df = w.left_join_csv('austin_animal_outcomes.csv', 'austin_animal_intakes.csv', 'merged_data.csv')

In [6]:
df.head()

,Animal ID,Name_x,DateTime_x,MonthYear_x,Date of Birth,Outcome Type,Outcome Subtype,Animal Type_x,Sex upon Outcome,Age upon Outcome,...,DateTime_y,MonthYear_y,Found Location,Intake Type,Intake Condition,Animal Type_y,Sex upon Intake,Age upon Intake,Breed_y,Color_y
0,A794011,Chunk,05/08/2019 06:20:00 PM,May 2019,05/02/2017,Rto-Adopt,NaN,Cat,Neutered Male,2 years,...,05/02/2019 04:51:00 PM,May 2019,Austin (TX),Owner Surrender,Normal,Cat,Neutered Male,2 years,Domestic Shorthair Mix,Brown Tabby/White
1,A776359,Gizmo,07/18/2018 04:02:00 PM,Jul 2018,07/12/2017,Adoption,NaN,Dog,Neutered Male,1 year,...,07/12/2018 12:46:00 PM,July 2018,7201 Levander Loop in Austin (TX),Stray,Normal,Dog,Intact Male,1 year,Chihuahua Shorthair Mix,White/Brown
2,A821648,NaN,08/16/2020 11:38:00 AM,Aug 2020,08/16/2019,Euthanasia,NaN,Other,Unknown,1 year,...,08/16/2020 10:10:00 AM,August 2020,Armadillo Rd And Clubway Ln in Austin (TX),Wildlife,Sick,Other,Unknown,1 year,Raccoon,Gray
3,A720371,Moose,02/13/2016 05:59:00 PM,Feb 2016,10/08/2015,Adoption,NaN,Dog,Neutered Male,4 months,...,02/08/2016 11:05:00 AM,February 2016,Dove Dr And E Stassney in Austin (TX),Stray,Normal,Dog,Intact Male,4 months,Anatol Shepherd/Labrador Retriever,Buff
4,A720371,Moose,02/13/2016 05:59:00 PM,Feb 2016,10/08/2015,Adoption,NaN,Dog,Neutered Male,4 months,...,02/15/2016 10:37:00 AM,February 2016,Austin (TX),Owner Surrender,Normal,Dog,Neutered Male,4 months,Anatol Shepherd/Labrador Retriever,Buff


In [7]:
df, model_df = w.get_prep_aa(df)

AttributeError: module 'wrangle' has no attribute 'get_prep_aa'

### Sneak Peak at the Data


In [ ]:
df.describe()

In [ ]:
model_df.describe()

### Summary of Data

In [ ]:
def plot_categorical_variables_1(df):
    for column in df.columns:
        if df[column].dtype == 'object':
            if len(df[column].unique()) > 10:
                   plt.figure(figsize=(12, 6))
            else:
                plt.figure()
                
            sns.countplot(df[column])
            plt.title(column)
            plt.xticks(rotation=45)
            plt.show()

# Example usage:
# Assuming 'data' is your DataFrame containing categorical variables
plot_categorical_variables_1(df)

In [ ]:
import plotly.express as px


def plot_categorical_variables_2(df):
    for column in df.columns:
        if df[column].dtype == 'object':
            fig = px.histogram(df, x=column)
            fig.update_traces(hovertemplate='Count: %{y}')
            fig.update_layout(title=column)
            fig.show()

# Example usage:
# Assuming 'data' is your DataFrame containing categorical variables
plot_categorical_variables_2(df)

In [ ]:
def plot_categorical_variable_3(model_df):
    for column in model_df.columns:
        if model_df[column].dtype == 'object':
            if len(model_df[column].unique()) > 10:
                   plt.figure(figsize=(12, 6))
            else:
                plt.figure()
                
            sns.countplot(model_df[column])
            plt.title(column)
            plt.xticks(rotation=45)
            plt.show()

# Example usage:
# Assuming 'data' is your DataFrame containing categorical variables
plot_categorical_variables_3(model_df)

In [ ]:
def plot_categorical_variables_4(model_df):
    for column in model_df.columns:
        if model_df[column].dtype == 'object':
            fig = px.histogram(model_df, x=column)
            fig.update_traces(hovertemplate='Count: %{y}')
            fig.update_layout(title=column)
            fig.show()

# Example usage:
# Assuming 'data' is your DataFrame containing categorical variables
plot_categorical_variables_4(model_df)

## Exploration

* Questions 1,2,3,


### Q . 

#### Observations: 


### Q 1

### Q 2

####  Observations: 


### Q 3 

####  Observations: 
*

### Q 4 

####  Observations: 


## Exploration Summary


## Report - Conclusion
### Summary


### Recommendations

* Provide additional takeaways or downloadable docs for extensively used topics
* We have data governance recommendations:
    - We found tens of thousands of rows with data that was missing and/or had nonsensical information
    * Collect information more information such as: 
        - Incidents (e.g. biting, abuse hx) 
        - Stated reason for return 
        - Reasons for turn in by owners
        - Vaccination status
        - Who turned the animal in (e.g. citizen, law enforcement, organization)

### Next Steps
* If provided more time we could have looked further into the additional two questions, and connected unknown users to cohorts